## 최소 공통 조상(Lowest Common Ancestor, LCA) 알고리즘
### 1. LCA 알고리즘이란
트리 구조에서 임의의 서로 다른 두 노드 A, B의 가장 가까운 공통 조상을 찾는 알고리즘
<img src="https://mblogthumb-phinf.pstatic.net/MjAxODA4MTZfMTUz/MDAxNTM0MzkxNjg1Mjk5.A9hYbAMasOzW7fNZWdog-12M_VbHalDrpmejBxPBp0Eg.9GxIE5iQ_qazG6nCkiGxdayYB2MJfepATp9bKPGjlcsg.PNG.jh20s/image.png?type=w800"><center>__그림 1. 트리 예시__</center>

### 단순한 방법
LCA를 구하고자 하는 두 노드들의 조상을 거슬러 올라감
1. 두 노드 A, B의 깊이가 같아질 때까지 더 깊은 노드 B를 부모로 이동
2. 두 노드 A, B가 같아질 때까지 두 노드의 부모로 이동

시간 복잡도는 일반적인 경우 $O(\log{N})$, 최악의 경우 $O(N)$

### 아이디어
1. 모든 노드에 대해 각 노드의 깊이와 $2^k$ 번째 조상을 모두 저장 (depth array, ancestor array)
2. 노드에서 부모로 한 칸씩 이동하지 않고, 한 번에 크게 이동
    - 두 노드 A, B의 깊이를 맞출 때, ancestor array를 참조하여 더 깊은 노드 B의 먼 조상부터 낮은 노드 A의 깊이와 비교하며 B의 조상이 A보다 깊거나 같으면 B를 이동
    - A와 B를 맞출 때, ancestor array를 참조하여 A와 B의 먼 조상부터 비교하며 두 노드의 조상이 다르면 A, B를 이동

| Index |  0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | ... | 16 | 17 |
|-------|----|---|---|---|---|---|---|---|---|---|----|-----|----|----|
| Value | -1 | 0 | 1 | 1 | 2 | 2 | 2 | 2 | 3 | 3 | 3  | ... | 5  | 5  |
<center>__표 1. 그림 1에 대한 depth array__</center>

| k | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | ... | 16 | 17 |
|---|---|---|---|---|---|---|---|---|---|---|----|-----|----|----|
| 0 | - | 0 | 1 | 1 | 2 | 2 | 3 | 3 | 4 | 4 | 5  | ... | 15 | 15 |
| 1 | - | - | - | - | 1 | 1 | 1 | 1 | 2 | 2 | 2  | ... | 12 | 12 |
| 2 | - | - | - | - | - | - | - | - | - | - | -  | ... | 3  | 3  |
| 3 | - | - | - | - | - | - | - | - | - | - | -  | ... | -  | -  |
<center>__표 2. 그림 1에 대한 ancestor array__</center>

### 구현
1. DFS를 이용한 트리 구성
~~~cpp
void getTree(int here, int parent) {
	depth[here] = depth[parent] + 1;
	ancestor[here][0] = parent;	// here의 2^(0)번째 조상, 즉 here의 부모
	
	// ancestor array 구성
	for (int i = 1; i <= MAX_LEVEL; i++) {
		int tmp = ancestor[here][i - 1];	// tmp = here의 2^(i-1)번째 조상
		// here의 2^(i)번째 조상 = tmp의 2^(i-1)번째 조상
		// ex) here의 2번째 조상 = tmp(here의 1번째 조상)의 1번째 조상
		ancestor[here][i] = ancestor[tmp][i - 1];
	}
	// here의 자식 노드에 대해 재귀 호출
	for (int i = 0; i < graph[here].size(); i++) {
		int there = graph[here][i]; // there = here와 연결된 i번째 노드
		if (there != parent)
			// ex) 1이 루트이고, 2와 3은 1의 자식, 4와 5는 2의 자식이면
			//		graph[1] = { 2, 3 }
			//		graph[2] = { 1, 4, 5 }
			//		graph[3] = { 1, ... } 이고,
			//     here = 2, there = 1일 때 there는 here의 부모이므로 제외
			getTree(there, here);
	}
}
~~~
2. depth array, ancestor array를 이용한 깊이 비교 및 노드 이동
~~~cpp
int lca(int a, int b) {
	// depth[a]와 depth[b]가 같지 않으면 depth[b]를 맞춰줌
	if (depth[a] != depth[b]) {
		if (depth[a] > depth[b])
			swap(a, b);
		for (int i = MAX_LEVEL; i >= 0; i--)
			if (depth[a] <= depth[ancestor[b][i]])
				b = ancestor[b][i];
	}
	// a와 b가 같은 수이면 자기 자신 리턴. ex) 편향 트리에서 LCA
	if (a == b) return a;
	// a와 b가 같지 않으면 두 노드의 공통 조상을 찾아 이동
	for (int i = MAX_LEVEL; i >= 0; i--)
		// 두 노드의 조상이 같지 않으면 같아질 때까지 이동
		if (ancestor[a][i] != ancestor[b][i])
			a = ancestor[a][i], b = ancestor[b][i];
    
	return ancestor[a][0];	// 공통 조상 리턴
}
~~~

### 참고 문제
1. [BOJ 11438번: LCA](https://www.acmicpc.net/problem/11437)
2. [BOJ 11438번: LCA 2](https://www.acmicpc.net/problem/11438)
3. [BOJ 1761번: 정점들의 거리](https://www.acmicpc.net/problem/1761)

---
### 참고 문제 1번 C++ Source Code
~~~cpp
#include <cstdio>
#include <vector>
#define MAX_NODE 50001
#define MAX_LEVEL	15
// 최대 노드 개수는 50,000 이고, 2^15 = 32,768, 2^16 = 65,536 이므로
// MAX_LEVEL을 16으로 설정하면 depth가 가장 큰 최악의 경우라도 (모든 노드의 자식이 1개인 편향 트리)
// 16번째 조상 탐색 시 루트 노드를 넘어가 버림 (65,536 > 50,000)
// 따라서 50,000 >= log_2(MAX_LEVEL)를 만족하는 15로 설정
using namespace std;

vector<int> graph[MAX_NODE];	// graph[x] = x와 연결된 모든 노드. graph[x][i] = x와 연결된 i번째 노드
int depth[MAX_NODE];	// depth[x] = 노드 x의 depth
int ancestor[MAX_NODE][MAX_LEVEL + 1];	// ancestor[x][y] = x의 2^y번째 조상

void swap(int &a, int &b) { int tmp = a; a = b; b = tmp; }
// graph를 이용하여 tree 만들기
void getTree(int here, int parent) {
	depth[here] = depth[parent] + 1;
	ancestor[here][0] = parent;	// here의 2^(0)번째 조상, 즉 here의 부모
	
	// ancestor array 구성
	for (int i = 1; i <= MAX_LEVEL; i++) {
		int tmp = ancestor[here][i - 1];	// tmp = here의 2^(i-1)번째 조상
		// here의 2^(i)번째 조상 = tmp의 2^(i-1)번째 조상
		// ex) here의 2번째 조상 = tmp(here의 1번째 조상)의 1번째 조상
		ancestor[here][i] = ancestor[tmp][i - 1];
	}
	// here의 자식 노드에 대해 재귀 호출
	for (int i = 0; i < graph[here].size(); i++) {
		int there = graph[here][i]; // there = here와 연결된 i번째 노드
		if (there != parent)
			// ex) 1이 루트이고, 2와 3은 1의 자식, 4와 5는 2의 자식이면
			//		graph[1] = { 2, 3 }
			//		graph[2] = { 1, 4, 5 }
			//		graph[3] = { 1, ... } 이고,
			//     here = 2, there = 1일 때 there는 here의 부모이므로 제외
			getTree(there, here);
	}
}
// 최소 공통 조상 탐색
int lca(int a, int b) {
	// depth[a]와 depth[b]가 같지 않으면 depth[b]를 맞춰줌
	if (depth[a] != depth[b]) {
		if (depth[a] > depth[b])
			swap(a, b);
		for (int i = MAX_LEVEL; i >= 0; i--)
			if (depth[a] <= depth[ancestor[b][i]])
				b = ancestor[b][i];
	}
	// a와 b가 같은 수이면 자기 자신 리턴. ex) 편향 트리에서 LCA
	if (a == b) return a;
	// a와 b가 같지 않으면 두 노드의 공통 조상을 찾아 이동
	for (int i = MAX_LEVEL; i >= 0; i--)
		// 두 노드의 조상이 같지 않으면 같아질 때까지 이동
		if (ancestor[a][i] != ancestor[b][i])
			a = ancestor[a][i], b = ancestor[b][i];

	return ancestor[a][0];	// 공통 조상 리턴
}

int main() {
	int N, M;
	scanf("%d", &N);
	// graph 만들기
	while (--N) {
		int from, to;
		scanf("%d %d", &from, &to);
		graph[from].push_back(to);
		graph[to].push_back(from);
	}
	depth[0] = -1;
	getTree(1, 0);
	scanf("%d", &M);
	while (M--) {
		int a, b;
		scanf("%d %d", &a, &b);
		printf("%d\n", lca(a, b));
	}
	return 0;
}
~~~